In [2]:
!pip install torch transformers pandas numpy faiss-cpu tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm  # Barra de progresso

# Tenta importar FAISS, senão usa fallback
try:
    import faiss
    USE_FAISS = True
except ImportError:
    USE_FAISS = False
    print("Aviso: FAISS não encontrado. Usando multiplicação de matrizes (mais lento, mas funciona).")

# ==========================================
# 1. CONFIGURAÇÕES E CARREGAMENTO
# ==========================================
MODEL_PATH = "./bert_original"  # Pasta onde você reconstruiu o modelo
BATCH_SIZE = 32              # Processar 32 textos por vez (ajuste conforme memória RAM)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Carregando modelo de: {MODEL_PATH}")
print(f"Dispositivo de processamento: {DEVICE.upper()}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModel.from_pretrained(MODEL_PATH).to(DEVICE)
model.eval() # Modo de avaliação (mais rápido, desliga dropout)

# ==========================================
# 2. FUNÇÕES OTIMIZADAS
# ==========================================

def generate_embeddings_batched(texts, tokenizer, model, batch_size=32):
    """Gera embeddings em lotes para não estourar a memória e ser rápido."""
    all_embeddings = []
    
    # Loop com barra de progresso
    for i in tqdm(range(0, len(texts), batch_size), desc="Gerando Embeddings"):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenização
        inputs = tokenizer(
            batch_texts, 
            padding=True, 
            truncation=True, 
            max_length=128, 
            return_tensors="pt"
        ).to(DEVICE)
        
        with torch.no_grad():
            outputs = model(**inputs)
            
        # Mean Pooling (Média da última camada)
        # Atenção: Mover para CPU para liberar VRAM/RAM
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        
        # Normalização L2 (Essencial para Cosseno funcionar com FAISS/MatMul)
        # Isso faz com que a Distância Euclidiana seja equivalente à Similaridade de Cosseno
        faiss.normalize_L2(embeddings) if USE_FAISS else None
        if not USE_FAISS:
            norm = np.linalg.norm(embeddings, axis=1, keepdims=True)
            embeddings = embeddings / (norm + 1e-10)
            
        all_embeddings.append(embeddings)
        
    return np.vstack(all_embeddings)

# ==========================================
# 3. PREPARAÇÃO DOS DADOS
# ==========================================

# --- A) Carregar seus Dataframes reais aqui ---
# df_categ = pd.read_csv("sua_base_categorias.csv")
# df_manif = pd.read_csv("sua_base_reclamacoes.csv")

# ... Para o exemplo, vou criar dados dummy na escala que você falou ...
print("Gerando dados de exemplo (40k manif, 5k categ)...")
df_categ = pd.DataFrame({
    'FAMILIA': ['Cartões'] * 100 + ['Conta'] * 100,
    'PRODUTO': ['Crédito'] * 100 + ['Corrente'] * 100,
    'ASSUNTO': [f'Problema {i}' for i in range(200)]
})
df_manif = pd.DataFrame({
    'ID': range(100),
    'TEXTO_CLIENTE': ["O app do banco travou e não consigo pagar boleto"] * 100
})

# --- B) Criar "Assinatura Hierárquica" das Categorias ---
print("Preparando hierarquia das categorias...")
df_categ['texto_full'] = df_categ['FAMILIA'] + " " + df_categ['PRODUTO'] + " " + df_categ['ASSUNTO']

# ==========================================
# 4. GERAÇÃO DOS VETORES (A parte pesada)
# ==========================================

print(f"\n1. Vetorizando {len(df_categ)} Categorias...")
categ_vectors = generate_embeddings_batched(df_categ['texto_full'].tolist(), tokenizer, model, BATCH_SIZE)

print(f"\n2. Vetorizando {len(df_manif)} Reclamações...")
manif_vectors = generate_embeddings_batched(df_manif['TEXTO_CLIENTE'].tolist(), tokenizer, model, BATCH_SIZE)

# ==========================================
# 5. BUSCA DE SIMILARIDADE (Otimização FAISS)
# ==========================================
print("\n3. Realizando o Matching (Cruzamento)...")

# Índices das melhores categorias
best_indices = []
confidence_scores = []

if USE_FAISS:
    # Cria um índice Flat (busca exata) com produto interno (IP)
    # Como os vetores estão normalizados, IP == Cosseno
    dimension = categ_vectors.shape[1]
    index = faiss.IndexFlatIP(dimension)
    
    # Adiciona as categorias ao "banco de dados" do FAISS
    index.add(categ_vectors)
    
    # Busca os 1 vizinhos mais próximos para cada reclamação
    # k=1 retorna a melhor categoria
    D, I = index.search(manif_vectors, k=1)
    
    best_indices = I.flatten()
    confidence_scores = D.flatten()

else:
    # Fallback: Multiplicação de Matrizes (Rápido, mas consome muita RAM)
    # Similaridade = A . B^T
    print("Usando Numpy Dot Product...")
    sim_matrix = np.dot(manif_vectors, categ_vectors.T)
    best_indices = np.argmax(sim_matrix, axis=1)
    confidence_scores = np.max(sim_matrix, axis=1)

# ==========================================
# 6. ATRIBUIÇÃO DOS RESULTADOS
# ==========================================

print("Atribuindo resultados ao dataframe...")

# Usamos os índices encontrados para buscar as linhas correspondentes no df_categ
matched_categories = df_categ.iloc[best_indices].reset_index(drop=True)

# Acopla ao dataframe original
df_final = pd.concat([df_manif, matched_categories[['FAMILIA', 'PRODUTO', 'ASSUNTO']]], axis=1)
df_final['SCORE_CONFIANCA'] = confidence_scores

# Opcional: Filtrar apenas classificações com confiança alta
# df_final = df_final[df_final['SCORE_CONFIANCA'] > 0.85]

print("\n--- AMOSTRA DO RESULTADO ---")
print(df_final[['TEXTO_CLIENTE', 'FAMILIA', 'PRODUTO', 'ASSUNTO', 'SCORE_CONFIANCA']].head())

# Salvar
# df_final.to_csv("resultado_classificacao.csv", index=False)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\felip\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\felip\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\felip\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", 

AttributeError: _ARRAY_API not found

Aviso: FAISS não encontrado. Usando multiplicação de matrizes (mais lento, mas funciona).
Carregando modelo de: ./bert_original
Dispositivo de processamento: CPU
Gerando dados de exemplo (40k manif, 5k categ)...
Preparando hierarquia das categorias...

1. Vetorizando 200 Categorias...


Gerando Embeddings: 100%|██████████| 7/7 [00:03<00:00,  1.97it/s]



2. Vetorizando 100 Reclamações...


Gerando Embeddings: 100%|██████████| 4/4 [00:01<00:00,  2.42it/s]


3. Realizando o Matching (Cruzamento)...
Usando Numpy Dot Product...
Atribuindo resultados ao dataframe...

--- AMOSTRA DO RESULTADO ---
                                      TEXTO_CLIENTE  FAMILIA  PRODUTO  \
0  O app do banco travou e não consigo pagar boleto  Cartões  Crédito   
1  O app do banco travou e não consigo pagar boleto  Cartões  Crédito   
2  O app do banco travou e não consigo pagar boleto  Cartões  Crédito   
3  O app do banco travou e não consigo pagar boleto  Cartões  Crédito   
4  O app do banco travou e não consigo pagar boleto  Cartões  Crédito   

      ASSUNTO  SCORE_CONFIANCA  
0  Problema 1          0.57154  
1  Problema 1          0.57154  
2  Problema 1          0.57154  
3  Problema 1          0.57154  
4  Problema 1          0.57154  


## Utilizando BERT + Histórico

In [3]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier # Rede neural leve
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib # Para salvar o modelo treinado

# ======================================================
# 1. CARREGAR DADOS E GERAR VETORES (Como antes)
# ======================================================
# Supondo que você já rodou o código anterior e tem a função 'generate_embeddings_batched'
# e o modelo BERT carregado.

# df_historico = pd.read_csv("historico_classificado.csv")
# Exemplo simulado:
df_historico = pd.DataFrame({
    'TEXTO_CLIENTE': [
        "cobrança indevida no cartão", "pix não chegou", "atendente gritou comigo", 
        "cartão não passa", "quero investir no cdb", "gerente destratou"
    ] * 100,
    'CATEGORIA_TARGET': [
        "Cartão - Cobrança", "Conta - Pix", "Atendimento - Postura", 
        "Cartão - Falha", "Investimento - Aplicação", "Atendimento - Postura"
    ] * 100
})

print("Gerando vetores do histórico (Isso o BERT faz bem)...")
X_vectors = generate_embeddings_batched(
    df_historico['TEXTO_CLIENTE'].tolist(), 
    tokenizer, 
    model
)
y_labels = df_historico['CATEGORIA_TARGET']

# ======================================================
# 2. TREINAR O CLASSIFICADOR (A Mágica)
# ======================================================
print("Treinando o cérebro da categorização...")

# Divide em treino e teste para validar se está bom
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y_labels, test_size=0.2, random_state=42)

# Usamos um MLPClassifier (Multi-Layer Perceptron).
# É uma rede neural simples que roda na CPU e aprende muito bem com vetores do BERT.
clf = MLPClassifier(
    hidden_layer_sizes=(256, 128), # Duas camadas de neurônios
    max_iter=500, 
    activation='relu', 
    solver='adam', 
    random_state=42,
    verbose=True
)

clf.fit(X_train, y_train)

# ======================================================
# 3. AVALIAR
# ======================================================
print("\nRelatório de Qualidade do Modelo:")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# ======================================================
# 4. SALVAR PARA USAR DEPOIS
# ======================================================
# Salva o classificador treinado (pesa poucos MBs)
joblib.dump(clf, 'meu_classificador_treinado.pkl')
print("Classificador salvo com sucesso!")

Gerando vetores do histórico (Isso o BERT faz bem)...


Gerando Embeddings: 100%|██████████| 19/19 [00:07<00:00,  2.51it/s]


Treinando o cérebro da categorização...
Iteration 1, loss = 1.59269702
Iteration 2, loss = 1.44265938
Iteration 3, loss = 1.29678345
Iteration 4, loss = 1.13392746
Iteration 5, loss = 0.95330161
Iteration 6, loss = 0.76430970
Iteration 7, loss = 0.58441502
Iteration 8, loss = 0.42137763
Iteration 9, loss = 0.28783959
Iteration 10, loss = 0.18811737
Iteration 11, loss = 0.11828376
Iteration 12, loss = 0.07396536
Iteration 13, loss = 0.04704050
Iteration 14, loss = 0.03078639
Iteration 15, loss = 0.02099532
Iteration 16, loss = 0.01498709
Iteration 17, loss = 0.01116718
Iteration 18, loss = 0.00863339
Iteration 19, loss = 0.00698980
Iteration 20, loss = 0.00583236
Iteration 21, loss = 0.00501481
Iteration 22, loss = 0.00440206
Iteration 23, loss = 0.00394072
Iteration 24, loss = 0.00358197
Iteration 25, loss = 0.00329869
Iteration 26, loss = 0.00306795
Iteration 27, loss = 0.00287312
Iteration 28, loss = 0.00270872
Iteration 29, loss = 0.00256682
Iteration 30, loss = 0.00244372
Iteration

In [ ]:
# Carrega o classificador que você treinou
clf_carregado = joblib.load('meu_classificador_treinado.pkl')

# 1. Carrega os novos dados (sem classificação)
df_novos = pd.read_csv("novas_reclamacoes.csv") 

# 2. Gera os vetores com o BERT (igual antes)
vetores_novos = generate_embeddings_batched(
    df_novos['TEXTO_CLIENTE'].tolist(), 
    tokenizer, 
    model
)

# 3. Pede para o classificador prever
predicoes = clf_carregado.predict(vetores_novos)

# 4. Salva no Excel
df_novos['CATEGORIA_PREDITA'] = predicoes